In [1]:
import asyncio
import websockets
import json

async def test_websocket():
    uri = "ws://localhost:8001/ws/assistant/?api_key=chave_teste"
    async with websockets.connect(uri) as websocket:
        message = {"message": "Qualquer coisa"}
        await websocket.send(json.dumps(message))
        response = await websocket.recv()
        print(f"Recebeu do servidor: {response}")

await test_websocket()


Recebeu do servidor: {"message": "Qualquer coisa"}


In [12]:
import asyncio
import websockets
import json

async def test_cart_consumer():
    uri = "ws://localhost:8001/ws/cart/?token=token_unico"
    async with websockets.connect(uri) as websocket:

        comida_id = 4

        add_message = {
            "action": "add",
            "comida_id": comida_id,
            "quantidade": 2
        }
        await websocket.send(json.dumps(add_message))
        response = await websocket.recv()
        print(f"Resposta ao adicionar: {response}")

        view_message = {"action": "view"}
        await websocket.send(json.dumps(view_message))
        response = await websocket.recv()
        print(f"Itens no carrinho: {response}")

        remove_message = {
            "action": "remove",
            "comida_id": comida_id
        }
        
        await websocket.send(json.dumps(remove_message))
        response = await websocket.recv()
        print(f"Resposta ao remover: {response}")
        await websocket.close()

await test_cart_consumer()


Resposta ao adicionar: {"status": "item adicionado"}
Itens no carrinho: {"cart": [{"comida__comida": "Salad", "quantidade": 1}]}
Resposta ao remover: {"status": "item removido"}


In [13]:
import django
import os


os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'core.settings')
django.setup()

from openai import OpenAI
from assistant.env import gpt_key
import time

class GPT:
    def __init__(self) -> None:
        self.client = OpenAI(api_key='sk-uKnINJx8t8sySAB2Np6jT3BIbkFJGIxTCalu9NzcsaXCmEcr')
        self.assistant_id = 'asst_HoVGULUxDroynUBGVzhF1Dry'
        
    def retrive_assistant(self):
        return self.client.beta.assistants.retrieve(self.assistant_id)
    
    def list_assistants(self):
        return self.client.beta.assistants.list(
            order='desc',
            limit='20'
        )
        
    def _create_thread(self):
        return self.client.beta.threads.create()
    
    def creat_message(self, message: str, thread_id):
        return self.client.beta.threads.messages.create(
            thread_id=thread_id,
            role='user',
            content=message
        )
    
    def run_assistant(self, thread_id, assistant_id):
        return self.client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id
        )
        
    def check_status_queue(self, thread_id, run_id):
        return self.client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )

    def show_response_assistant(self, message_user):
        assistant = self.retrive_assistant()
        thread = self._create_thread()
        message = self.creat_message(message_user, thread.id)
        run = self.run_assistant(thread.id, assistant.id)
        
        while run.status != 'completed':
            time.sleep(1)
        
        return run


gpt = GPT()

teste = gpt.show_response_assistant('Olá, quais opções de hambuguer estão disponíveis?')

print(teste)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-uKnIN***************************************mEcr. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}